In [5]:
import cv2
import numpy as np

In [2]:
def setup_video(video_path, duration):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    num_frames = int(duration * fps)
    frames = [None] * num_frames
    
    for i in range(num_frames):
        ret, frame = cap.read()
        if not ret:
            break
        frames[i] = frame
    
    cap.release()
    return frames, fps

In [3]:
def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), cv2.BORDER_DEFAULT)
    _, thresh = cv2.threshold(blur,220, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    midpoints = []
    for cnt in contours:
        M = cv2.moments(cnt)
        if M['m00'] != 0:
            cx, cy = int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])
            cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 2, (0, 0, 255), -1)
            midpoints.append((cx, cy))
    return midpoints

In [4]:
# Setup
name = input("Enter name: ")
plane = input("Enter plane(r for right, l for left): ")
video_path = rf"E:\Gait Analysis NMH\Video\{name.capitalize()}{plane.capitalize()}.mp4"
user_input = input("Press 'a' for simpler method or 'b' for exceptional method: ")
frames, fps = setup_video(video_path, 30)

frame_index = 1
while frame_index < len(frames):
    frame = frames[frame_index].copy()
    midpoints = process_frame(frame)
    
    # Sort midpoints
    sorted_y = sorted(midpoints, key=lambda x: x[1])
    if user_input == 'a':
        last_two = sorted_y[-2:]
        if plane == 'l':
            sorted_x = sorted(last_two, reverse=True, key=lambda x: x[0])
        elif plane == 'r':
            sorted_x = sorted(last_two, key=lambda x: x[0])
        final_points = sorted_y[:-2] + sorted_x
    elif user_input == 'b':
        final_points = sorted_y[:-2] + sorted_y[-2:]

    # Process frame if enough points
    if len(final_points) == 5:
        slopes = [(final_points[i + 1][1] - final_points[i][1]) / (final_points[i + 1][0] - final_points[i][0]) 
                  if final_points[i + 1][0] != final_points[i][0] else np.inf 
                  for i in range(4)]
        for i in range(4):
            cv2.line(frame, final_points[i], final_points[i + 1], (0, 0, 255), 2)
    else:
        cv2.putText(frame, "Insufficient points detected", (30, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1)

    # Display frame number
    cv2.putText(frame, f"Frame: {frame_index}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)
    cv2.imshow('Frame', frame)

    # Handle key actions
    key = cv2.waitKey(0) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('f'):
        frame_index = (frame_index + 1) % len(frames)
    elif key == ord('b'):
        frame_index = (frame_index - 1) % len(frames)

cv2.destroyAllWindows()